In [0]:
#!!pip uninstall tensorflow
#!pip install tensorflow==2.1.0


In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
PROJECT_DIR = "/content/drive/My Drive/Colab Notebooks/cleansed_data.csv"
DATA_FILE_NAME = 'My Drive/Colab Notebooks/input.xlsx'
CLEANSED_FILE_DIR = '/My Drive/Colab Notebooks/'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
import keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Activation, Bidirectional, Flatten, BatchNormalization
from tensorflow.keras.models import Model, Sequential

Using TensorFlow backend.


In [0]:
def load_doc(filename):
  if(filename.endswith('.xlsx')):
    data_df = pd.read_excel(filename, lines=True)
  elif(filename.endswith('.csv')):
    data_df = pd.read_csv(filename, keep_default_na = False)
  return data_df

In [0]:
def encode_target(target):
  le = LabelEncoder()
  return le.fit_transform(target), le

def decode_prediction(pred, encoder):
  return encoder.inverse_transform(pred)

In [9]:
cleansed_data_df = load_doc(PROJECT_DIR)
print('shape of Data : ', cleansed_data_df.shape)

shape of Data :  (8500, 4)


In [10]:
cleansed_data_df.describe()

,short_description,description,combined_desc,assignment_group
count,8500,8500,8500,8500
unique,6000,5484,5634,74
top,job job failed job scheduler,,,GRP_0
freq,473,1123,1047,3976


In [11]:
cleansed_data_df.head()

,short_description,description,combined_desc,assignment_group
0,login issue,verified user details employee manager checked...,login issue verified user details employee man...,GRP_0
1,outlook,team meetings skype meetings etc not appearing...,outlook team meetings skype meetings etc not a...,GRP_0
2,cant log vpn,hi cannot log vpn,cant log vpn hi cannot log vpn,GRP_0
3,unable access hr tool page,unable access hr tool page,unable access hr tool page unable access hr to...,GRP_0
4,skype error,skype error,skype error skype error,GRP_0


In [0]:
labelencoder = LabelEncoder()
cleansed_data_df['assignment_group'] = labelencoder.fit_transform(cleansed_data_df['assignment_group'])

In [64]:
cleansed_data_df

,short_description,description,combined_desc,assignment_group
0,login issue,verified user details employee manager checked...,login issue verified user details employee man...,0
1,outlook,team meetings skype meetings etc not appearing...,outlook team meetings skype meetings etc not a...,0
2,cant log vpn,hi cannot log vpn,cant log vpn hi cannot log vpn,0
3,unable access hr tool page,unable access hr tool page,unable access hr tool page unable access hr to...,0
4,skype error,skype error,skype error skype error,0
...,...,...,...,...
8495,emails not coming zz mail,good afternoon not receiving emails sent zz ma...,emails not coming zz mail good afternoon not r...,22
8496,telephony software issue,telephony software issue,telephony software issue telephony software issue,0
8497,windows password reset tifpdchb pedxruyf,windows password reset tifpdchb pedxruyf,windows password reset tifpdchb pedxruyf windo...,0
8498,machine funcionando,unable access machine utilities finish drawers...,machine funcionando unable access machine util...,59


In [0]:
from sklearn.model_selection import train_test_split

# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(cleansed_data_df['combined_desc'], cleansed_data_df['assignment_group'], test_size=0.33, shuffle= True)

In [66]:
y_train.head()


2176    45
933      5
7578     0
4202    11
5442     0
Name: assignment_group, dtype: int64

In [68]:
print(x_train[22])

unable connect vpn unable connect vpn


In [69]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.utils import to_categorical


y_train_cate = to_categorical(y_train)
y_test_cate = to_categorical(y_valid)
print(y_train_cate.shape)

x_train = x_train.astype(str)
x_valid = x_valid.astype(str)

#Tokenize the sentences
tokenizer = Tokenizer(num_words=10000,char_level=False)

#preparing vocabulary
tokenizer.fit_on_texts(x_train)

#converting text into integer sequences
x_tr_seq  = tokenizer.texts_to_sequences(x_train) 
x_val_seq = tokenizer.texts_to_sequences(x_valid)

#padding to prepare sequences of same length
x_tr_seq  = pad_sequences(x_tr_seq, maxlen=200)
x_val_seq = pad_sequences(x_val_seq, maxlen=200)


(5695, 74)


In [71]:
print(x_train[22])
print(x_tr_seq[22])
#print(x_train[24])
#print(x_tr_seq[22])
#print(x_tr_seq[24])
#print(x_tr_seq[25])

unable connect vpn unable connect vpn
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0  45 238  11  45
 238  11]


In [72]:
y_train = np.asarray(y_train)
print(y_train.shape)
type(y_train)

(5695,)


numpy.ndarray

**Pretrained Glove embeddings with 100d**

In [73]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('../content/drive/My Drive/Colab Notebooks/glove.6B.100d.txt')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [74]:
size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

7814


In [0]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((size_of_vocabulary, 100))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [0]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import *

In [77]:
model=Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,100,weights=[embedding_matrix],input_length=200,trainable=True)) 

#lstm layer
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2)))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(128,activation='relu'))  
model.add(Dense(74,activation='softmax'))  

#Add loss function, metrics, optimizer
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=["acc"]) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  

#Print summary of model
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 200, 100)          781400    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 200, 256)          234496    
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_8 (Dense)              (None, 74)                9546      
Total params: 1,058,338
Trainable params: 1,058,338
Non-trainable params: 0
_________________________________________________________________
None


In [78]:
history = model.fit(x_tr_seq,y_train,batch_size=32,epochs=10,validation_data=(x_val_seq,y_valid),verbose=1,callbacks=[es,mc])

Train on 5695 samples, validate on 2805 samples
Epoch 1/10
5695/5695 [==============================] - 110s 19ms/step - loss: 2.3004 - acc: 0.5198 - val_loss: 1.9488 - val_acc: 0.5558

Epoch 00001: val_acc improved from -inf to 0.55579, saving model to best_model.h5
Epoch 2/10
5695/5695 [==============================] - 109s 19ms/step - loss: 1.7865 - acc: 0.5777 - val_loss: 1.7954 - val_acc: 0.5843

Epoch 00002: val_acc improved from 0.55579 to 0.58431, saving model to best_model.h5
Epoch 3/10
5695/5695 [==============================] - 108s 19ms/step - loss: 1.5725 - acc: 0.6174 - val_loss: 1.6393 - val_acc: 0.6018

Epoch 00003: val_acc improved from 0.58431 to 0.60178, saving model to best_model.h5
Epoch 4/10
5695/5695 [==============================] - 108s 19ms/step - loss: 1.3931 - acc: 0.6386 - val_loss: 1.5727 - val_acc: 0.6164

Epoch 00004: val_acc improved from 0.60178 to 0.61640, saving model to best_model.h5
Epoch 5/10
5695/5695 [==============================] - 108s 19

In [79]:
#loading best model
from keras.models import load_model
model = load_model('best_model.h5')

#evaluation 
_,val_acc = model.evaluate(x_val_seq,y_valid, batch_size=32)
print(val_acc)

2805/2805 [==============================] - 9s 3ms/step
0.6285204887390137


In [52]:
pip install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.8.3)


In [0]:
from sklearn.model_selection import train_test_split

# Split the data
x_train_vec, x_valid_vec, y_train_vec, y_valid_vec = train_test_split(cleansed_data_df['combined_desc'], cleansed_data_df['assignment_group'], test_size=0.33, shuffle= True)

In [91]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

y_train_cate = to_categorical(y_train_vec)
y_test_cate = to_categorical(y_train_vec)
print(y_train_cate.shape)

x_train_vec = x_train_vec.astype(str)
x_valid_vec = x_valid_vec.astype(str)

#Tokenize the sentences
tokenizer = Tokenizer(num_words=10000,char_level=False)
#preparing vocabulary
tokenizer.fit_on_texts(x_train_vec)

#converting text into integer sequences
x_tr_seq_vec  = tokenizer.texts_to_sequences(x_train_vec) 
x_val_seq_vec = tokenizer.texts_to_sequences(x_valid_vec)

#padding to prepare sequences of same length
x_tr_seq_vec  = pad_sequences(x_tr_seq_vec, maxlen=200)
x_val_seq_vec = pad_sequences(x_val_seq_vec, maxlen=200)



(5695, 74)


In [92]:
size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

8065


In [0]:
from gensim.models.keyedvectors import KeyedVectors
PROJECT_DIR = "/content/drive/My Drive/Colab Notebooks/"
EMBEDDING_FILE = 'SO_vectors_200.bin'
EmbeddingFile = PROJECT_DIR+EMBEDDING_FILE
word2vec = KeyedVectors.load_word2vec_format(EmbeddingFile, binary=True)
MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 200000


In [0]:
# create a weight matrix for words in training docs
EMBEDDING_DIM = 200
embedding_matrix_word2vec = np.zeros((size_of_vocabulary, EMBEDDING_DIM))

for word, i in tokenizer.word_index.items():
    if word in word2vec.vocab:
        embedding_matrix_word2vec[i] = word2vec.word_vec(word)   

In [95]:
model=Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,200,weights=[embedding_matrix_word2vec],input_length=200,trainable=True)) 

#lstm layer
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2)))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(128,activation='relu'))  
model.add(Dense(74,activation='softmax'))  

#Add loss function, metrics, optimizer
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=["acc"]) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  

#Print summary of model
print(model.summary())


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 200, 200)          1613000   
_________________________________________________________________
bidirectional_7 (Bidirection (None, 200, 256)          336896    
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 256)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_14 (Dense)             (None, 74)                9546      
Total params: 1,992,338
Trainable params: 1,992,338
Non-trainable params: 0
_________________________________________________________________
None


In [96]:
history = model.fit(x_tr_seq_vec,y_train_vec,batch_size=32,epochs=10,validation_data=(x_val_seq_vec,y_valid_vec),verbose=1,callbacks=[es,mc])

Train on 5695 samples, validate on 2805 samples
Epoch 1/10
5695/5695 [==============================] - 154s 27ms/step - loss: 2.1597 - acc: 0.5387 - val_loss: 1.7324 - val_acc: 0.5925

Epoch 00001: val_acc improved from -inf to 0.59251, saving model to best_model.h5
Epoch 2/10
5695/5695 [==============================] - 155s 27ms/step - loss: 1.6080 - acc: 0.6042 - val_loss: 1.5298 - val_acc: 0.6271

Epoch 00002: val_acc improved from 0.59251 to 0.62709, saving model to best_model.h5
Epoch 3/10
5695/5695 [==============================] - 154s 27ms/step - loss: 1.3883 - acc: 0.6427 - val_loss: 1.4607 - val_acc: 0.6399

Epoch 00003: val_acc improved from 0.62709 to 0.63993, saving model to best_model.h5
Epoch 4/10
5695/5695 [==============================] - 154s 27ms/step - loss: 1.2304 - acc: 0.6715 - val_loss: 1.4076 - val_acc: 0.6474

Epoch 00004: val_acc improved from 0.63993 to 0.64742, saving model to best_model.h5
Epoch 5/10
5695/5695 [==============================] - 154s 27

In [98]:
#loading best model
from keras.models import load_model
model = load_model('best_model.h5')

#evaluation 
_,val_acc = model.evaluate(x_val_seq_vec,y_valid_vec, batch_size=32)
print(val_acc)

2805/2805 [==============================] - 16s 6ms/step
0.6552584767341614


**FASTTEXT pre trained embedding**

In [0]:
from sklearn.model_selection import train_test_split

# Split the data
x_train_fast, x_valid_fast, y_train_fast, y_valid_fast = train_test_split(cleansed_data_df['combined_desc'], cleansed_data_df['assignment_group'], test_size=0.33, shuffle= True)


In [105]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

y_train_cate = to_categorical(y_train_fast)
y_test_cate = to_categorical(y_valid_fast)
print(y_train_cate.shape)

x_train_fast = x_train_fast.astype(str)
x_valid_fast = x_valid_fast.astype(str)

#Tokenize the sentences
tokenizer = Tokenizer(num_words=10000,char_level=False)

#preparing vocabulary
tokenizer.fit_on_texts(list(x_train_fast))

#converting text into integer sequences
x_tr_seq_fast  = tokenizer.texts_to_sequences(x_train_fast) 
x_val_seq_fast = tokenizer.texts_to_sequences(x_valid_fast)

#padding to prepare sequences of same length
x_tr_seq_fast  = pad_sequences(x_tr_seq_fast, maxlen=300)
x_val_seq_fast = pad_sequences(x_val_seq_fast, maxlen=300)

size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)


(5695, 74)
7947


In [0]:
max_features = 200000 #17780 #using all unique words
embedding_dim = 300
num_classes = 74
batch_size = 32

embedding_path = "/content/drive/My Drive/Colab Notebooks/wiki-news-300d-1M.vec"

def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

word_index = tokenizer.word_index
nb_words = len(word_index)
embedding_matrix_fast = np.zeros((nb_words + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None: embedding_matrix_fast[i] = embedding_vector

In [108]:
model=Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,300,weights=[embedding_matrix_fast],input_length=300,trainable=True)) 

#lstm layer
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2)))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(128,activation='relu'))  
model.add(Dense(74,activation='softmax'))  

#Add loss function, metrics, optimizer
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=["acc"]) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  

#Print summary of model
print(model.summary())

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 300, 300)          2384100   
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 300, 256)          439296    
_________________________________________________________________
global_max_pooling1d_10 (Glo (None, 256)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_20 (Dense)             (None, 74)                9546      
Total params: 2,865,838
Trainable params: 2,865,838
Non-trainable params: 0
_________________________________________________________________
None


In [109]:
model.fit(x_tr_seq_fast,y_train_fast,batch_size=32,epochs=10,validation_data=(x_val_seq_fast,y_valid_fast),verbose=1,callbacks=[es,mc])

Train on 5695 samples, validate on 2805 samples
Epoch 1/10
5695/5695 [==============================] - 284s 50ms/step - loss: 2.2602 - acc: 0.5250 - val_loss: 1.8556 - val_acc: 0.5768

Epoch 00001: val_acc improved from -inf to 0.57683, saving model to best_model.h5
Epoch 2/10
5695/5695 [==============================] - 285s 50ms/step - loss: 1.6641 - acc: 0.6000 - val_loss: 1.6567 - val_acc: 0.6029

Epoch 00002: val_acc improved from 0.57683 to 0.60285, saving model to best_model.h5
Epoch 3/10
5695/5695 [==============================] - 284s 50ms/step - loss: 1.3629 - acc: 0.6637 - val_loss: 1.6104 - val_acc: 0.6185

Epoch 00003: val_acc improved from 0.60285 to 0.61854, saving model to best_model.h5
Epoch 4/10
5695/5695 [==============================] - 283s 50ms/step - loss: 1.1398 - acc: 0.7096 - val_loss: 1.6238 - val_acc: 0.6178

Epoch 00004: val_acc did not improve from 0.61854
Epoch 5/10
5695/5695 [==============================] - 283s 50ms/step - loss: 0.9520 - acc: 0.761

In [115]:
#loading best model
from keras.models import load_model
model = load_model('best_model.h5')

#evaluation 
_,val_acc = model.evaluate(x_val_seq_fast,y_valid_fast, batch_size=32)
print(val_acc)

2805/2805 [==============================] - 26s 9ms/step
0.6263814568519592
